# Amazon Product Data Analysis

## Dataset Source
Kaggle - Amazon Products Dataset

## Objective
The purpose of this project is to analyze Amazon product data to uncover insights 
related to product categories, pricing strategies, brand competition, and customer behavior.

In [3]:
# Import necessary libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
# Load dataset

df = pd.read_csv('data/amazon_product.csv')

# Review the dataset structure and information

In [5]:
df.head()   

,Unnamed: 0,asin,product_title,product_price,product_original_price,currency,product_star_rating,product_num_ratings,product_url,product_photo,...,is_best_seller,is_amazon_choice,is_prime,climate_pledge_friendly,sales_volume,delivery,has_variations,product_availability,unit_price,unit_count
0,0,B0BQ118F2T,Moto G Play 2023 3-Day Battery Unlocked Made f...,$99.99,$169.99,USD,4.0,2929,https://www.amazon.com/dp/B0BQ118F2T,https://m.media-amazon.com/images/I/61K1Fz5Lxv...,...,False,False,True,False,6K+ bought in past month,"FREE delivery Tue, Aug 6",True,NaN,NaN,NaN
1,1,B0CTD47P22,"SAMSUNG Galaxy A15 5G (SM-156M/DSN), 128GB 6GB...",$149.74,$158.00,USD,4.2,135,https://www.amazon.com/dp/B0CTD47P22,https://m.media-amazon.com/images/I/51QhB2CfqS...,...,False,False,True,False,3K+ bought in past month,"FREE delivery Wed, Aug 7 Only 7 left in stock ...",False,Only 7 left in stock - order soon.,NaN,NaN
2,2,B0CHH6X6H2,Total by Verizon | Samsung Galaxy A03s | Locke...,$49.88,NaN,USD,3.9,205,https://www.amazon.com/dp/B0CHH6X6H2,https://m.media-amazon.com/images/I/812woqv69C...,...,False,False,True,False,2K+ bought in past month,"FREE delivery Tue, Aug 6",False,NaN,NaN,NaN
3,3,B0BZ9XNBRB,Google Pixel 7a - Unlocked Android Cell Phone ...,$335.00,$499.00,USD,4.3,2248,https://www.amazon.com/dp/B0BZ9XNBRB,https://m.media-amazon.com/images/I/61r7cCpQPl...,...,False,False,False,False,10K+ bought in past month,FREE delivery Aug 6 - 8,True,NaN,NaN,NaN
4,4,B0CN1QSH8Q,"SAMSUNG Galaxy A15 5G A Series Cell Phone, 128...",$199.99,NaN,USD,4.1,423,https://www.amazon.com/dp/B0CN1QSH8Q,https://m.media-amazon.com/images/I/61s0ZzwzSC...,...,False,False,True,True,3K+ bought in past month,"FREE delivery Tue, Aug 6",True,NaN,NaN,NaN


In [7]:
df.columns

Index(['Unnamed: 0', 'asin', 'product_title', 'product_price',
       'product_original_price', 'currency', 'product_star_rating',
       'product_num_ratings', 'product_url', 'product_photo',
       'product_num_offers', 'product_minimum_offer_price', 'is_best_seller',
       'is_amazon_choice', 'is_prime', 'climate_pledge_friendly',
       'sales_volume', 'delivery', 'has_variations', 'product_availability',
       'unit_price', 'unit_count'],
      dtype='object')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 22 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   64 non-null     int64  
 1   asin                         64 non-null     object 
 2   product_title                64 non-null     object 
 3   product_price                64 non-null     object 
 4   product_original_price       27 non-null     object 
 5   currency                     64 non-null     object 
 6   product_star_rating          54 non-null     float64
 7   product_num_ratings          64 non-null     int64  
 8   product_url                  64 non-null     object 
 9   product_photo                64 non-null     object 
 10  product_num_offers           64 non-null     int64  
 11  product_minimum_offer_price  64 non-null     object 
 12  is_best_seller               64 non-null     bool   
 13  is_amazon_choice      

In [8]:
# Check for missing values
df.isnull().sum()


Unnamed: 0                      0
asin                            0
product_title                   0
product_price                   0
product_original_price         37
currency                        0
product_star_rating            10
product_num_ratings             0
product_url                     0
product_photo                   0
product_num_offers              0
product_minimum_offer_price     0
is_best_seller                  0
is_amazon_choice                0
is_prime                        0
climate_pledge_friendly         0
sales_volume                    3
delivery                        1
has_variations                  0
product_availability           63
unit_price                     59
unit_count                     59
dtype: int64

In [14]:
# Drop rows with missing values
df = df.dropna()


In [15]:
print(df.columns)


Index(['Unnamed: 0', 'asin', 'product_title', 'product_price',
       'product_original_price', 'currency', 'product_star_rating',
       'product_num_ratings', 'product_url', 'product_photo',
       'product_num_offers', 'product_minimum_offer_price', 'is_best_seller',
       'is_amazon_choice', 'is_prime', 'climate_pledge_friendly',
       'sales_volume', 'delivery', 'has_variations', 'product_availability',
       'unit_price', 'unit_count'],
      dtype='object')


In [17]:
# -------------------------------
# 1. Clean Column Names
# -------------------------------

# Remove leading/trailing spaces in column names
df.columns = df.columns.str.strip()

# Optional: rename columns for convenience
df.rename(columns={
    'product_price': 'price',
    'product_original_price': 'original_price',
    'product_minimum_offer_price': 'min_offer_price',
    'product_star_rating': 'rating',
    'product_num_ratings': 'num_reviews'
}, inplace=True)

# -------------------------------
# 2. Convert Price Columns to Numeric
# -------------------------------

# List of columns to convert
price_columns = ['price', 'original_price', 'min_offer_price']

for col in price_columns:
    # Remove '$' and ',' using raw string (avoids warnings)
    df[col] = df[col].replace(r'[\$,]', '', regex=True)
    
    # Convert to float, invalid parsing becomes NaN
    df[col] = pd.to_numeric(df[col], errors='coerce')

# -------------------------------
# 3. Convert Ratings & Reviews to Numeric
# -------------------------------

df['rating'] = pd.to_numeric(df['rating'], errors='coerce')
df['num_reviews'] = pd.to_numeric(df['num_reviews'], errors='coerce')

# -------------------------------
# 4. Drop Rows with Missing Essential Values
# -------------------------------

df = df.dropna(subset=['price', 'rating', 'num_reviews'])

# -------------------------------
# 5. Check the Cleaned Data
# -------------------------------

print(df[['price','original_price','min_offer_price','rating','num_reviews']].head())
print(df.info())


Empty DataFrame
Columns: [price, original_price, min_offer_price, rating, num_reviews]
Index: []
<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 22 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               0 non-null      int64  
 1   asin                     0 non-null      object 
 2   product_title            0 non-null      object 
 3   price                    0 non-null      float64
 4   original_price           0 non-null      int64  
 5   currency                 0 non-null      object 
 6   rating                   0 non-null      float64
 7   num_reviews              0 non-null      int64  
 8   product_url              0 non-null      object 
 9   product_photo            0 non-null      object 
 10  product_num_offers       0 non-null      int64  
 11  min_offer_price          0 non-null      int64  
 12  is_best_seller           0 non-null      bool   
 13  

In [20]:
category_counts = df['category'].value_counts()
print("Most products by category:")
print(category_counts.head(10))




KeyError: 'category'


1. # Which Amazon product categories have the most products?
2. # Which categories have the highest average prices?
3. What is the overall distribution of product prices?
4. Is there a relationship between price and rating?
5. # Which brands appear most frequently?
6. # Which brands have the highest ratings?
7. Which products have the most reviews?
8. Which products provide the best value (high rating & low price)?
9. Which categories show opportunities for new sellers?


In [18]:
top_categories = df['category'].value_counts().head(10)

plt.figure()
top_categories.plot(kind='bar')
plt.title("Top 10 Product Categories on Amazon")
plt.xlabel("Category")
plt.ylabel("Number of Products")
plt.xticks(rotation=45)
plt.show()


KeyError: 'category'

In [ ]:
business_questions = [
    "Which ASINs/products account for the highest recent sales volume, and how do their prices and discounts compare?",
    "Does Prime / Best Seller / Amazon Choice status significantly increase sales volume or average price?",
    "How do product star ratings and number of ratings correlate with sales volume and price?",
    "Which products have the largest discounts (product_original_price vs product_price), and do discounts drive higher sales?",
    "Do delivery promises (e.g., FREE delivery, fast delivery, 'Only X left') or product_availability affect sales volume?",
    "For items with unit_price/unit_count or multiple offers, which offer the best cost-per-unit and does lower unit price relate to higher sales?"
]